In [45]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [46]:
DIR_KNOWNS = 'knowns'




######### MOBILENET

In [47]:
# Leer mobilenet_graph.pb
with tf.io.gfile.GFile('mobilenet_graph.pb','rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as mobilenet:
    tf.import_graph_def(graph_def,name='')
print("si")

si


In [48]:
print(mobilenet)

In [49]:
def load_image(DIR, NAME):
    with tf.io.gfile.GFile('mobilenet_graph.pb', 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as mobilenet:
        tf.import_graph_def(graph_def, name='')
    print("si")

    return cv2.cvtColor(cv2.imread(f'{DIR}/{name}'), cv2.COLOR_BGR2RGB)
print("si")



si


In [50]:
def detect_faces(image, score_threshold=0.7):
    global boxes, scores
    (imh, imw) = image.shape[:-1]
    img = np.expand_dims(image,axis=0)

    # Inicializar mobilenet
    sess = tf.compat.v1.Session(graph=mobilenet)
    image_tensor = mobilenet.get_tensor_by_name('image_tensor:0')
    boxes = mobilenet.get_tensor_by_name('detection_boxes:0')
    scores = mobilenet.get_tensor_by_name('detection_scores:0')

    # Predicción (detección)
    (boxes, scores) = sess.run([boxes, scores], feed_dict={image_tensor:img})

    # Reajustar tamaños boxes, scores
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)

    # Depurar bounding boxes
    idx = np.where(scores>=score_threshold)[0]

    # Crear bounding boxes
    bboxes = []
    for index in idx:
        ymin, xmin, ymax, xmax = boxes[index,:]
        (left, right, top, bottom) = (xmin*imw, xmax*imw, ymin*imh, ymax*imh)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        bboxes.append([left,right,top,bottom])

    return bboxes
print("si")

si


In [51]:

# Dibujar bounding boxes
def draw_box(image,box,color,line_width=6):
    if box==[]:
        return image
    else:
        cv2.rectangle(image,(box[0],box[2]),(box[1],box[3]),color,line_width)
    return image
print("si")

si


In [52]:
# Extraer rostros
def extract_faces(image,bboxes,new_size=(160,160)):
    cropped_faces = []
    for box in bboxes:
        left, right, top, bottom = box
        face = image[top:bottom,left:right]
        cropped_faces.append(cv2.resize(face,dsize=new_size))
    return cropped_faces
print("si")



##########    FACENET


si


In [53]:
facenet = load_model('facenet_keras.h5')
print(facenet.input_shape)
print(facenet.output_shape)

(None, 160, 160, 3)
(None, 128)


In [54]:
def compute_embedding(model,face):
    face = face.astype('float32')

    mean, std = face.mean(), face.std()
    face = (face-mean) / std

    face = np.expand_dims(face,axis=0)

    embedding = model.predict(face)
    return embedding

In [55]:
# Embeddings referencia
known_embeddings = []

print('Procesando rostros conocidos...')
for name in os.listdir(DIR_KNOWNS):
    if name.endswith('.jpg'):
        print(f'   {name}')
        image = load_image(DIR_KNOWNS,name)
        bboxes = detect_faces(image)
        face = extract_faces(image,bboxes)
        known_embeddings.append(compute_embedding(facenet,face[0]))


print("si")

Procesando rostros conocidos...
   ABOMZO_01.jpg
si
   ADMIN.jpg
si
   ANJU_AIKYAN_01.jpg
si
   ESTADISTICA.jpg
si
   HUARACCNAL_01.jpg
si
   JAIGAN_01.jpg
si
   kanako_01.jpg
si
   kanako_02.jpg
si
   kanako_03.jpg
si
   kanako_04.jpg
si
   kanako_05.jpg
si
   NONICVIERZO_01.jpg
si
   NONIC_01.jpg
si
   NONIC_02.jpg
si
   NONIC_03.jpg
si
   NONIC_04.jpg
si
   NONIC_05.jpg
si
   TEO_01.jpg
si
si


In [61]:
known_embeddings

[array([[ 0.63805735,  0.27963048,  0.6774217 , -1.1008509 ,  0.20627193,
          1.672714  ,  0.6101894 , -1.1200795 , -0.32650012, -0.7418546 ,
         -1.0412803 ,  1.088603  ,  0.8217519 , -0.16355385,  0.43651557,
          0.7598792 ,  1.0396626 , -2.3248124 ,  0.6696172 , -1.467506  ,
         -0.28508618,  0.27176213, -1.2659066 , -0.607004  ,  0.465914  ,
         -1.866787  ,  0.7800417 ,  0.96040106, -0.6050037 ,  0.28955913,
          0.48691282, -0.18939638,  0.8953096 , -0.99468285,  0.09836826,
         -1.1851922 , -1.0137239 , -0.68365407,  0.6377441 , -0.451153  ,
         -1.1425046 , -0.04775747, -1.8920797 , -0.7192921 ,  0.41743305,
         -1.2493663 , -1.5844536 ,  1.2301658 ,  0.8387128 ,  0.30429256,
          0.16343698,  0.8044244 ,  0.5582228 , -0.565622  ,  1.3294178 ,
          0.48113495, -0.3445217 , -1.8296479 , -0.01892981,  1.2107927 ,
         -0.07988644, -0.39018154,  1.4181309 ,  0.9648003 ,  0.33434638,
         -0.6870712 ,  0.18311134,  0.

In [57]:
len(known_embeddings)



18

In [58]:
bin_unlabeled_database=open('unlabeled_database.pickle','wb')
pickle.dump(known_embeddings,bin_unlabeled_database)
bin_unlabeled_database.close()


In [59]:
if os.path.getsize('unlabeled_database.pickle')>0:
    with open('unlabeled_database.pickle','rb') as f:
        unpickler=pickle.Unpickler(f)
        scores=unpickler.load()

len(scores)

18